In [88]:
%pip install spacy pandas typer


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: /nvme/geckos/venvs/jupyterlab-3.12/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import sys
import json
import pandas as pd
import spacy
from spacy import displacy 
from spacy.matcher import Matcher
from pathlib import Path
import operator as op

from IPython.display import display, Markdown as Md


In [55]:
files = Path("./output/").glob("*.json")
data = [{**json.load(f.open()), "file": f.name} for f in files]
# display(data)


df = pd.DataFrame(data, columns=["tags", "descrip", "title", "url", "file", "metadata"])
df

,tags,descrip,title,url,file,metadata
0,"[go, golangprojects]",\n\n\nSenior Golang backend developer at Cloud...,Golang / Go Job: Senior Golang backend developer,https://www.golangprojects.com/golang-go-job-g...,Golang___Go_Job:_Senior_Golang_backend_develop...,{'remote': 'Americas'}
1,"[go, golangprojects]",\n\n\nStaff Software Engineer (Golang) at Stre...,Golang / Go Job: Staff Software Engineer (Golang),https://www.golangprojects.com/golang-go-job-g...,Golang___Go_Job:_Staff_Software_Engineer_(Gola...,{'remote': 'Amsterdam European'}
2,"[go, golangprojects]",\n\n\nSenior Software Engineer – Golang at Clo...,Golang / Go Job: Senior Software Engineer – Go...,https://www.golangprojects.com/golang-go-job-g...,Golang___Go_Job:_Senior_Software_Engineer_–_Go...,{}
3,"[go, golangprojects]",\n\n\nBackend Software Engineer (Golang) at In...,Golang / Go Job: Backend Software Engineer (Go...,https://www.golangprojects.com/golang-go-job-g...,Golang___Go_Job:_Backend_Software_Engineer_(Go...,{}
4,"[go, golangprojects]","\n\n\nSoftware Engineer, Backend at Glue\n\nRe...","Golang / Go Job: Software Engineer, Backend",https://www.golangprojects.com/golang-go-job-g...,"Golang___Go_Job:_Software_Engineer,_Backend.json",{'remote': 'Americas'}
...,...,...,...,...,...,...
134,[indeed],Profile insightsFind out how your skills align...,Senior VBA Developer,https://www.indeed.com/rc/clk?jk=35706498cac80...,Senior_VBA_Developer.json,{}
135,[indeed],About Vercel:\nVercel’s Frontend Cloud provide...,"Software Engineer, Lifecycle",https://www.indeed.com/rc/clk?jk=a9cb697f6d1d0...,"Software_Engineer,_Lifecycle.json",{}
136,[indeed],Our client is a hardcore team that creates imm...,Backend Tech Lead,https://www.indeed.com/rc/clk?jk=411707c0270d0...,Backend_Tech_Lead.json,{}
137,[functionalworks],"Join one of Web3's hottest projects, Douro Lab...",Senior Software Engineer (Applied Cryptography),https://functional.works-hub.com/jobs/remote-s...,Senior_Software_Engineer_(Applied_Cryptography...,{'remote': 'Porto Portugal'}


In [56]:
df.query("metadata.isna()")

,tags,descrip,title,url,file,metadata
138,[indeed],Profile insightsFind out how your skills align...,Node Js,https://www.indeed.com/rc/clk?jk=b6d8a9da14512...,Node_Js.json,NaN


In [68]:
# Replace any NaN in metadata with a default empty dict
df.metadata = df.metadata.apply(lambda x: {} if pd.isna(x) else x)

In [70]:
df.query("metadata.isna()")

,tags,descrip,title,url,file,metadata


In [61]:
# install the spacy model 
!{sys.executable} -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 10.0 MB/s eta 0:00:00m eta 0:00:010:00:01

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: /nvme/geckos/venvs/jupyterlab-3.12/bin/python -m pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [4]:
nlp = spacy.load("en_core_web_sm")

In [74]:
location_mat = Matcher(nlp.vocab)
location_pats = [
    [{"ENT_TYPE": "LOC"}],
    [{"ENT_TYPE": "NORP"}],
    [{"ENT_TYPE": "GPE"}],
    [{"ENT_TYPE": "TIME"}],
    [{"LOWER": "worldwide"}],
]
location_mat.add("LOCATION", location_pats)
remote_mat = Matcher(nlp.vocab)
remote_pats = [
    [{"LOWER": "remote"}],
]
remote_mat.add("REMOTE", remote_pats)

In [75]:
def collect_matcher(m, doc, window=0):
    return [doc[s-window:e+window].as_doc() for _, s, e in m(doc)]

def add_remote_metadata(row):
    # Do not overwrite the scrapper
    if "remote" in row.metadata:
        return row.metadata
        
    doc = nlp(row.descrip)
    # Find remote word
    remotes = collect_matcher(remote_mat, doc, 5)
    if remotes:
        locations = set(str(l).strip() for doc in remotes for l in collect_matcher(location_mat, doc))
        if locations:
            return {
                **row.metadata,
                "remote": " ".join(sorted(map(str, locations)))).strip()
            }
    return row.metadata

df["metadata"] = df.apply(add_remote_metadata, axis=1)
df

,tags,descrip,title,url,file,metadata
0,"[go, golangprojects]",\n\n\nSenior Golang backend developer at Cloud...,Golang / Go Job: Senior Golang backend developer,https://www.golangprojects.com/golang-go-job-g...,Golang___Go_Job:_Senior_Golang_backend_develop...,{'remote': 'AmericasAmericas Americas'}
1,"[go, golangprojects]",\n\n\nStaff Software Engineer (Golang) at Stre...,Golang / Go Job: Staff Software Engineer (Golang),https://www.golangprojects.com/golang-go-job-g...,Golang___Go_Job:_Staff_Software_Engineer_(Gola...,{'remote': 'Amsterdam EuropeanAmsterdam Europe...
2,"[go, golangprojects]",\n\n\nSenior Software Engineer – Golang at Clo...,Golang / Go Job: Senior Software Engineer – Go...,https://www.golangprojects.com/golang-go-job-g...,Golang___Go_Job:_Senior_Software_Engineer_–_Go...,{}
3,"[go, golangprojects]",\n\n\nBackend Software Engineer (Golang) at In...,Golang / Go Job: Backend Software Engineer (Go...,https://www.golangprojects.com/golang-go-job-g...,Golang___Go_Job:_Backend_Software_Engineer_(Go...,{}
4,"[go, golangprojects]","\n\n\nSoftware Engineer, Backend at Glue\n\nRe...","Golang / Go Job: Software Engineer, Backend",https://www.golangprojects.com/golang-go-job-g...,"Golang___Go_Job:_Software_Engineer,_Backend.json",{'remote': 'AmericasAmericas Americas'}
...,...,...,...,...,...,...
134,[indeed],Profile insightsFind out how your skills align...,Senior VBA Developer,https://www.indeed.com/rc/clk?jk=35706498cac80...,Senior_VBA_Developer.json,{}
135,[indeed],About Vercel:\nVercel’s Frontend Cloud provide...,"Software Engineer, Lifecycle",https://www.indeed.com/rc/clk?jk=a9cb697f6d1d0...,"Software_Engineer,_Lifecycle.json",{}
136,[indeed],Our client is a hardcore team that creates imm...,Backend Tech Lead,https://www.indeed.com/rc/clk?jk=411707c0270d0...,Backend_Tech_Lead.json,{}
137,[functionalworks],"Join one of Web3's hottest projects, Douro Lab...",Senior Software Engineer (Applied Cryptography),https://functional.works-hub.com/jobs/remote-s...,Senior_Software_Engineer_(Applied_Cryptography...,{'remote': 'Porto PortugalPorto Portugal Porto...


In [76]:
for _, row in df.iterrows():
    with open(f"output/{row.file}", "w") as f:
        f.write(json.dumps({ "title": row.title,
                              "descrip": row.descrip,
                            "url": row.url,
                            "tags": row.tags,
                            "metadata": row.metadata}))
        print(f"File {f.name} written")

File output/Golang___Go_Job:_Senior_Golang_backend_developer.json written
File output/Golang___Go_Job:_Staff_Software_Engineer_(Golang).json written
File output/Golang___Go_Job:_Senior_Software_Engineer_–_Golang.json written
File output/Golang___Go_Job:_Backend_Software_Engineer_(Golang).json written
File output/Golang___Go_Job:_Software_Engineer,_Backend.json written
File output/Golang___Go_Job:_Senior_Software_Engineer.json written
File output/Golang___Go_Job:_Golang_Developer_(Ninja_Team).json written
File output/Golang___Go_Job:_Golang_Engineer.json written
File output/Golang___Go_Job:_Platform_Engineer_(Golang).json written
File output/Golang___Go_Job:_Senior_Engineer,_Golang.json written
File output/Golang___Go_Job:_Senior_Golang_Developer.json written
File output/Golang___Go_Job:_Senior_Developer,_Back-End,_Golang_(Greece).json written
File output/Senior_Rust_Engineer.json written
File output/Web3_Backend_Engineer_(Rust).json written
File output/Rust_Engineer_-_Generative_AI_Ima